# Loss function zoo

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

## Dice Loss

In [ ]:
#PyTorch
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)

        #flatten label and prediction tensors
        # inputs = inputs.view(-1)
        # targets = targets.view(-1)
        if inputs.size(1) > 1:  # Multiclass case
            targets = torch.nn.functional.one_hot(targets, num_classes=inputs.size(1)).float()
        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

# Example code
#Keras
# def DiceLoss(targets, inputs, smooth=1e-6):

#     #flatten label and prediction tensors
#     inputs = K.flatten(inputs)
#     targets = K.flatten(targets)

#     intersection = K.sum(K.dot(targets, inputs))
#     dice = (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
#     return 1 - dice

## BCE-Dice Loss

In [ ]:
class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)

        #flatten label and prediction tensors
        # inputs = inputs.view(-1)
        # targets = targets.view(-1)
        if inputs.size(1) > 1:  # Multiclass case
            targets = torch.nn.functional.one_hot(targets, num_classes=inputs.size(1)).float()

        intersection = (inputs * targets).sum()
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss

        return Dice_BCE


# Example code
# Keras
# def DiceBCELoss(targets, inputs, smooth=1e-6):

#     #flatten label and prediction tensors
#     inputs = K.flatten(inputs)
#     targets = K.flatten(targets)

#     BCE =  binary_crossentropy(targets, inputs)
#     intersection = K.sum(K.dot(targets, inputs))
#     dice_loss = 1 - (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
#     Dice_BCE = BCE + dice_loss

#     return Dice_BCE

## Jaccard/Intersection over Union (IoU) Loss

In [ ]:
#PyTorch
class IoULoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(IoULoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)

        #flatten label and prediction tensors
        # inputs = inputs.view(-1)
        # targets = targets.view(-1)
        if inputs.size(1) > 1:  # Multiclass case
            targets = torch.nn.functional.one_hot(targets, num_classes=inputs.size(1)).float()

        #intersection is equivalent to True Positive count
        #union is the mutually inclusive area of all labels & predictions
        intersection = (inputs * targets).sum()
        total = (inputs + targets).sum()
        union = total - intersection

        IoU = (intersection + smooth)/(union + smooth)

        return 1 - IoU

# Example code
# Keras
#Keras
# def IoULoss(targets, inputs, smooth=1e-6):

#     #flatten label and prediction tensors
#     inputs = K.flatten(inputs)
#     targets = K.flatten(targets)

#     intersection = K.sum(K.dot(targets, inputs))
#     total = K.sum(targets) + K.sum(inputs)
#     union = total - intersection

#     IoU = (intersection + smooth) / (union + smooth)
#     return 1 - IoU

## Focal Loss

In [ ]:
#PyTorch
ALPHA = 0.8
GAMMA = 2

class FocalLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(FocalLoss, self).__init__()

    def forward(self, inputs, targets, alpha=0.8, gamma=2, smooth=1):
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)
        #flatten label and prediction tensors
        # inputs = inputs.view(-1)
        # targets = targets.view(-1)

        if inputs.size(1) > 1:  # Multiclass case
            targets = torch.nn.functional.one_hot(targets, num_classes=inputs.size(1)).float()

        # print(inputs.size(), targets.size())
        #first compute binary cross-entropy
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        BCE_EXP = torch.exp(-BCE)
        focal_loss = alpha * (1-BCE_EXP)**gamma * BCE

        return focal_loss

# Example code
# Keras
#Keras
# ALPHA = 0.8
# GAMMA = 2

# def FocalLoss(targets, inputs, alpha=ALPHA, gamma=GAMMA):

#     inputs = K.flatten(inputs)
#     targets = K.flatten(targets)

#     BCE = K.binary_crossentropy(targets, inputs)
#     BCE_EXP = K.exp(-BCE)
#     focal_loss = K.mean(alpha * K.pow((1-BCE_EXP), gamma) * BCE)

#     return focal_loss

## Tversky Loss

In [ ]:
#PyTorch
ALPHA = 0.5
BETA = 0.5

class TverskyLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(TverskyLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1, alpha=0.5, beta=0.5):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)

        #flatten label and prediction tensors
        # inputs = inputs.view(-1)
        # targets = targets.view(-1)
        if inputs.size(1) > 1:  # Multiclass case
            targets = torch.nn.functional.one_hot(targets, num_classes=inputs.size(1)).float()

        #True Positives, False Positives & False Negatives
        TP = (inputs * targets).sum()
        FP = ((1-targets) * inputs).sum()
        FN = (targets * (1-inputs)).sum()

        Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)

        return 1 - Tversky

# Example code
# Keras
#Keras
# ALPHA = 0.5
# BETA = 0.5

# def TverskyLoss(targets, inputs, alpha=ALPHA, beta=BETA, smooth=1e-6):

#         #flatten label and prediction tensors
#         inputs = K.flatten(inputs)
#         targets = K.flatten(targets)

#         #True Positives, False Positives & False Negatives
#         TP = K.sum((inputs * targets))
#         FP = K.sum(((1-targets) * inputs))
#         FN = K.sum((targets * (1-inputs)))

#         Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)

#         return 1 - Tversky

## Focal Tversky Loss

In [ ]:
#PyTorch
ALPHA = 0.5
BETA = 0.5
GAMMA = 1

class FocalTverskyLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(FocalTverskyLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1, alpha=0.5, beta=0.5, gamma=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)

        #flatten label and prediction tensors
        # inputs = inputs.view(-1)
        # targets = targets.view(-1)

        if inputs.size(1) > 1:  # Multiclass case
            targets = torch.nn.functional.one_hot(targets, num_classes=inputs.size(1)).float()

        #True Positives, False Positives & False Negatives
        TP = (inputs * targets).sum()
        FP = ((1-targets) * inputs).sum()
        FN = (targets * (1-inputs)).sum()

        Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)
        FocalTversky = (1 - Tversky)**gamma

        return FocalTversky

# Example code
# Keras
#Keras
# ALPHA = 0.5
# BETA = 0.5
# GAMMA = 1

# def FocalTverskyLoss(targets, inputs, alpha=ALPHA, beta=BETA, gamma=GAMMA, smooth=1e-6):

#         #flatten label and prediction tensors
#         inputs = K.flatten(inputs)
#         targets = K.flatten(targets)

#         #True Positives, False Positives & False Negatives
#         TP = K.sum((inputs * targets))
#         FP = K.sum(((1-targets) * inputs))
#         FN = K.sum((targets * (1-inputs)))

#         Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)
#         FocalTversky = K.pow((1 - Tversky), gamma)

#         return FocalTversky

## Lovasz Hinge Loss

## Combo Loss

In [ ]:
#PyTorch
ALPHA = 0.5 # < 0.5 penalises FP more, > 0.5 penalises FN more
CE_RATIO = 0.5 #weighted contribution of modified CE loss compared to Dice loss

class ComboLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(ComboLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1, alpha=0.5, beta=0.5, eps=1e-9):

        #flatten label and prediction tensors
        # inputs = inputs.view(-1)
        # targets = targets.view(-1)

        if inputs.size(1) > 1:  # Multiclass case
            targets = torch.nn.functional.one_hot(targets, num_classes=inputs.size(1)).float()

        #True Positives, False Positives & False Negatives
        intersection = (inputs * targets).sum()
        dice = (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)

        inputs = torch.clamp(inputs, eps, 1.0 - eps)
        out = - (ALPHA * ((targets * torch.log(inputs)) + ((1 - ALPHA) * (1.0 - targets) * torch.log(1.0 - inputs))))
        weighted_ce = out.mean(-1)
        combo = (CE_RATIO * weighted_ce) - ((1 - CE_RATIO) * dice)

        return combo

# class ComboLoss(nn.Module):
#     def __init__(self, smooth=1, alpha=0.5, beta=0.5, ce_ratio=0.5):
#         super(ComboLoss, self).__init__()
#         self.smooth = smooth
#         self.alpha = alpha
#         self.beta = beta
#         self.ce_ratio = ce_ratio

#     def forward(self, inputs, targets):
#         eps = 1e-9

#         # One-hot encode targets for multiclass
#         if inputs.size(1) > 1:  # Multiclass case
#             targets = torch.nn.functional.one_hot(targets, num_classes=inputs.size(1)).permute(0, 3, 1, 2).float()

#         # Ensure inputs are probabilities
#         inputs = torch.clamp(inputs, eps, 1.0 - eps)

#         # True Positives, False Positives, and False Negatives
#         intersection = (inputs * targets).sum(dim=(2, 3))  # Sum over spatial dimensions
#         dice = (2. * intersection + self.smooth) / (inputs.sum(dim=(2, 3)) + targets.sum(dim=(2, 3)) + self.smooth)
#         dice_loss = 1 - dice.mean()  # Mean over batch

#         # Cross-Entropy Loss
#         weighted_ce = -((self.alpha * targets * torch.log(inputs)) +
#                         ((1 - self.alpha) * (1 - targets) * torch.log(1 - inputs)))
#         weighted_ce = weighted_ce.mean(dim=(2, 3)).mean(dim=0)  # Mean over spatial and batch dimensions

#         # Combine losses
#         combo = (self.ce_ratio * weighted_ce) + ((1 - self.ce_ratio) * dice_loss)
#         return combo


# Example code
# Keras
#Keras
# ALPHA = 0.5 # < 0.5 penalises FP more, > 0.5 penalises FN more
# CE_RATIO = 0.5 #weighted contribution of modified CE loss compared to Dice loss

# def Combo_loss(targets, inputs, eps=1e-9):
#     targets = K.flatten(targets)
#     inputs = K.flatten(inputs)

#     intersection = K.sum(targets * inputs)
#     dice = (2. * intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
#     inputs = K.clip(inputs, eps, 1.0 - eps)
#     out = - (ALPHA * ((targets * K.log(inputs)) + ((1 - ALPHA) * (1.0 - targets) * K.log(1.0 - inputs))))
#     weighted_ce = K.mean(out, axis=-1)
#     combo = (CE_RATIO * weighted_ce) - ((1 - CE_RATIO) * dice)

#     return combo

# Experiment loss function with Deep Learning model

## Pre-Processing

In [ ]:
# tiền xử lý dữ liệu và xử lý mất cân bằng
def _convert_format(data, target):
    import pandas as pd
    data.replace(',', '.', regex=True, inplace=True)

    # Convert all object columns to numeric, where possible
    for col in data.select_dtypes(include='object').columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')
    # convert target column to object
    data[target] = data[target].astype(str)
    # data[target] = data[target].astype('category')

    print('finished converting data to correct format')
    # Display the updated dataset info
    return data


def _normalization(data):
    import pandas as pd
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
    print('finished normalization')

    return data


def _handle_missing_value(data):
    data.fillna(data.median(), inplace=True)
    print('finished filling missing values')

    return data


def _handle_class_imbalance(data):
    from imblearn.over_sampling import ADASYN
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    ada = ADASYN(random_state=42)
    X_resampled, y_resampled = ada.fit_resample(X, y)
    print('finished balancing data')

    return X_resampled, y_resampled


def pre_processing(data, target,pytorch=None):
    import torch
    import pandas as pd
    data = _convert_format(data, target)
    # data = _normalization(data)
    # data = _handle_missing_value(data)
    # X_resampled, y_resampled = _handle_class_imbalance(data)
    print('finished pre-processing data')

    # X = data.iloc[:, :-1]
    # y = data.iloc[:, -1]

    if pytorch:
        X_tensor = torch.tensor(X_resampled.values, dtype=torch.float32)
        y_tensor = torch.tensor(y_resampled.values, dtype=torch.float32)
        return X_tensor, y_tensor

    # return X_resampled, y_resampled
    return data


In [ ]:
# data_path = "Advanced-DA-Task-management-cleaned.csv"
# # merge X and y into one
# df_resample = pd.concat([X, y], axis=1)
# df_resample.to_csv(data_path, index=False)

In [ ]:
import pandas as pd

data_path = "Advanced-DA-Task-management-cleaned.csv"
target_col = "R/Not-R"
df = pd.read_csv(data_path)
# X, y = pre_processing(df, target_col, pytorch=False)
cleaned_df = pre_processing(df, target_col, pytorch=False)

finished converting data to correct format
finished pre-processing data


In [ ]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2623 entries, 0 to 2622
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EBIT           2623 non-null   float64
 1   IE             2623 non-null   float64
 2   EBIT - IE      2623 non-null   float64
 3   WC/TA          2623 non-null   float64
 4   TL/TA          2623 non-null   float64
 5   EBIT/IE        2623 non-null   float64
 6   NOCF/CL        2623 non-null   float64
 7   RE/TA          2623 non-null   float64
 8   NP/S           2623 non-null   float64
 9   EBIT/TA        2623 non-null   float64
 10  S/TA           2623 non-null   float64
 11  TA(t)/TA(t-1)  2623 non-null   float64
 12  NP(t)/NP(t-1)  2623 non-null   float64
 13  R/Not-R        2623 non-null   object 
dtypes: float64(13), object(1)
memory usage: 287.0+ KB


## Model Zoo

### Install neccesity packages

In [ ]:
!pip install "pytorch_tabular[extra]" -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.4/891.4 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.9 MB/s eta 0:00:00


In [ ]:
!pip install "pytorch_tabular" -q

In [ ]:
# from pytorch_tabular.utils import load_covertype_dataset
# data, _, _, _ = load_covertype_dataset()
# data, cat_col_names, num_col_names, target_col = load_covertype_dataset()


In [ ]:
from rich import print
# One of the easiest ways to identify categorical features is using the pandas select_dtypes function.
# categorical_features = data.select_dtypes(include=['object'])
# print(categorical_features.columns)

data = cleaned_df.copy()
# cat_col_names = X.select_dtypes(include=['object'])
cat_col_names = []
num_col_names = data.select_dtypes(include=['float64'])
target_col = "R/Not-R"

In [ ]:
# Sửa chỗ này

# This separation have already been done for you while loading this particular dataset from `PyTorch Tabular`. Let's load the dataset in the right way.
# data, cat_col_names, num_col_names, target_col = load_covertype_dataset()
# Let's also print out a few details
print(f"Data Shape: {data.shape} | # of cat cols: {len(cat_col_names)} | # of num cols: {len(num_col_names.columns.tolist())}")
print(f"[bold dodger_blue2] Features: {num_col_names.columns.tolist() + cat_col_names}[/bold dodger_blue2]")
print(f"[bold purple4]Target: {target_col}[/bold purple4]")

Data Shape: (2623, 14) | # of cat cols: 0 | # of num cols: 13

 Features: ['EBIT', 'IE', 'EBIT - IE', 'WC/TA', 'TL/TA', 'EBIT/IE', 'NOCF/CL', 'RE/TA', 'NP/S', 'EBIT/TA', 'S/TA', 
'TA(t)/TA(t-1)', 'NP(t)/NP(t-1)']

Target: R/Not-R

In [ ]:
# Let's also check the data for missing values
print(data.isna().sum())

EBIT             0
IE               0
EBIT - IE        0
WC/TA            0
TL/TA            0
EBIT/IE          0
NOCF/CL          0
RE/TA            0
NP/S             0
EBIT/TA          0
S/TA             0
TA(t)/TA(t-1)    0
NP(t)/NP(t-1)    0
R/Not-R          0
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, random_state=42, test_size=0.2)
train, val = train_test_split(train, random_state=42, test_size=0.2)
print(f"Train Shape: {train.shape} | Val Shape: {val.shape} | Test Shape: {test.shape}")

Train Shape: (1678, 14) | Val Shape: (420, 14) | Test Shape: (525, 14)

In [ ]:
DiceLoss()
DiceBCELoss()
IoULoss()
FocalLoss()
TverskyLoss()
FocalTverskyLoss()

FocalTverskyLoss()

### Node(Neural Oblivious Decision Ensembles for Deep Learning on Tabular Data)(Huyền)

In [ ]:
from pytorch_tabular.models import NodeConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

node_data_config = DataConfig(
    target=[
        target_col
    ],  # target should always be a list
    continuous_cols=num_col_names.columns.tolist(),
    # categorical_cols=cat_col_names,
)

node_config = NodeConfig(
    task="classification",               # Task: 'classification' or 'regression'
    num_layers=3,                        # Number of layers in the architecture ### Sửa cái này được
    num_trees=512,                       # Number of trees in each layer ### Sửa cái này được
    additional_tree_output_dim=2,        # Additional output dimensions passed across layers ### Sửa cái này được
    depth=6,                             # Depth of each tree
    choice_function="entmax15",          # Choice function for soft feature selection: 'entmax15' or 'sparsemax'
    bin_function="entmoid15",            # Bin function for sparse leaf weights: 'entmoid15' or 'sparsemoid'
    max_features=None,                   # Limit on features carried forward; default = None
    input_dropout=0.1,                   # Dropout applied to inputs between layers
    initialize_response="normal",        # Initialization method for response variable
    initialize_selection_logits="uniform",  # Initialization for feature selectors
    threshold_init_beta=1.0,             # Beta parameter for data-aware threshold initialization
    threshold_init_cutoff=1.0,           # Cutoff for threshold initialization
    embedding_dims=None,                 # Infer embedding dimensions automatically
    embedding_dropout=0.1,               # Dropout for categorical embeddings
    batch_norm_continuous_input=True,    # Batch normalization for continuous features
    learning_rate=1e-3,                  # Learning rate
    metrics=["accuracy", "precision", "recall", "f1_score", "auroc"],
    metrics_prob_input=[True, True, True, True, True],           # Input probabilities for accuracy metric
    seed=42                              # Seed for reproducibility
)

node_optimizer_config = OptimizerConfig()

node_trainer_config = TrainerConfig(
    auto_lr_find=False,
    batch_size=512, # có thể để batch nhỏ vì dữ liệu nhỏ
    max_epochs=100, # có thể để nhu này và giảm batch vì dữ liệu nhỏ
)


In [ ]:
from pytorch_tabular import TabularModel

tabular_model = TabularModel(
    data_config=node_data_config,
    model_config=node_config,
    optimizer_config=node_optimizer_config,
    trainer_config=node_trainer_config,
    verbose=True
)

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off


In [ ]:
# DiceLoss() (done)
# DiceBCELoss() (done)
# IoULoss() (done)
# FocalLoss() (done)
# TverskyLoss() (done)
# FocalTverskyLoss() (done)

'''
Đổi loss function ở đây
Test với trường hợp không sử dụng custom loss function(ko cần truyền tham số)
'''

# tabular_model.fit(train=train, validation=val)
tabular_model.fit(train=train, validation=val, loss=DiceLoss())

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for classification task
INFO:pytorch_tabular.tabular_model:Preparing the Model: NODEModel
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/models/node/node_model.py:111: UserWarning: Ignoring head config because NODE has a specific head which subsets the tree outputs
  warnings.warn("Ignoring head config because NODE has a specific head which subsets the tree outputs")
INFO:pytorch_tabular.models.node.node_model:Data Aware Initialization of NODE using a forward pass with 2000 batch size....
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_tab

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ _backbone        │ NODEBackbone     │ 19.4 M │ train │
│ 1 │ _embedding_layer │ Embedding1dLayer │     26 │ train │
│ 2 │ _head            │ Lambda           │      0 │ train │
│ 3 │ loss             │ CrossEntropyLoss │      0 │ train │
└───┴──────────────────┴──────────────────┴────────┴───────┘

Trainable params: 19.4 M                                                                                           
Non-trainable params: 2.3 K                                                                                        
Total params: 19.4 M                                                                                               
Total estimated model params size (MB): 77                                                                         
Modules in train mode: 12                                                                                          
Modules in eval mode: 0

Output()

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (4)
is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you 
want to see logs for the training epoch.

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
pred_df = tabular_model.predict(test)
pred_df.head()

In [ ]:
# - Accuracy
# - Precision
# - Recall
# - F1-score(F-measure)
# - AUC(auroc)

result = tabular_model.evaluate(test)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5009523630142212     │
│        test_auroc         │    0.5654933452606201     │
│       test_f1_score       │    0.5009523630142212     │
│         test_loss         │    0.4888763129711151     │
│        test_loss_0        │    0.4888763129711151     │
│      test_precision       │    0.5009523630142212     │
│        test_recall        │    0.5009523630142212     │
└───────────────────────────┴───────────────────────────┘

### TabNet(Huyền)

In [ ]:
from pytorch_tabular.models import TabNetModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

tabnet_data_config = DataConfig(
    target=[
        target_col
    ],  # target should always be a list
    continuous_cols=num_col_names.columns.tolist(),
    # categorical_cols=cat_col_names,
)

tabnet_config = TabNetModelConfig(
    n_d=16,                          # Dimension of prediction layer
    n_a=16,                          # Dimension of attention layer
    n_steps=5,                       # Number of successive steps in the network
    gamma=1.5,                       # Scaling factor for attention updates
    n_independent=2,                 # Number of independent GLU layers
    n_shared=2,                      # Number of shared GLU layers
    virtual_batch_size=128,          # Batch size for Ghost Batch Normalization
    mask_type="entmax",              # Masking function: 'sparsemax' or 'entmax'
    task="classification",           # Task type: classification or regression
    learning_rate=1e-3,              # Learning rate
    metrics=["accuracy", "precision", "recall", "f1_score", "auroc"],  # Metrics to track
    metrics_prob_input=[True, True, True, True, True],           # Probabilities as input
    seed=42                          # Seed for reproducibility
)

tabnet_optimizer_config = OptimizerConfig()

tabnet_trainer_config = TrainerConfig(
    auto_lr_find=False,
    batch_size=512,
    max_epochs=100,
)




In [ ]:
from pytorch_tabular import TabularModel

tabular_model = TabularModel(
    data_config=tabnet_data_config,
    model_config=tabnet_config,
    optimizer_config=tabnet_optimizer_config,
    trainer_config=tabnet_trainer_config,
    verbose=True
)

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off


In [ ]:
# DiceLoss()
# DiceBCELoss()
# IoULoss()
# FocalLoss()
# TverskyLoss()
# FocalTverskyLoss()

'''
Đổi loss function ở đây
Test với trường hợp không sử dụng custom loss function(ko cần truyền tham số)
'''
tabular_model.fit(train=train, validation=val, loss=DiceBCELoss())

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for classification task
INFO:pytorch_tabular.tabular_model:Preparing the Model: TabNetModel
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_tabular.tabular_model:Training Started
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type           ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ custom_loss      │ DiceBCELoss    │      0 │ train │
│ 1 │ _embedding_layer │ Identity       │      0 │ train │
│ 2 │ _backbone        │ TabNetBackbone │ 31.8 K │ train │
│ 3 │ _head            │ Identity       │      0 │ train │
└───┴──────────────────┴────────────────┴────────┴───────┘

Trainable params: 31.8 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 31.8 K                                                                                               
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 155                                                                                         
Modules in eval mode: 0

Output()

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


INFO:pytorch_tabular.tabular_model:Training the model completed
INFO:pytorch_tabular.tabular_model:Loading the best model


In [ ]:
pred_df = tabular_model.predict(test)
pred_df.head()

,R/Not-R_0.0_probability,R/Not-R_1.0_probability,R/Not-R_prediction
856,0.299910,0.700090,1.0
1867,0.495577,0.504423,1.0
651,0.878879,0.121121,0.0
368,0.390712,0.609288,1.0
2465,0.613174,0.386826,0.0


In [ ]:
result = tabular_model.evaluate(test)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5180952548980713     │
│        test_auroc         │    0.5262024402618408     │
│       test_f1_score       │    0.5180952548980713     │
│         test_loss         │    1.3075473308563232     │
│        test_loss_0        │    1.3075473308563232     │
│      test_precision       │    0.5180952548980713     │
│        test_recall        │    0.5180952548980713     │
└───────────────────────────┴───────────────────────────┘

### Mixture Density Networks (bỏ)

In [ ]:
from pytorch_tabular.models import MDNConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

mdn_data_config = DataConfig(
    target=[
        target_col
    ],  # target should always be a list
    continuous_cols=num_col_names.columns.tolist(),
    categorical_cols=cat_col_names,
)

mixture_density_config = MDNConfig(
    backbone_config_class="FTTransformerConfig",  # Example backbone configuration
    backbone_config_params={"d_token": 32, "n_blocks": 3},  # Parameters for the backbone
    task="classification",          # Set task to classification
    # head="MixtureDensityHead",      # Specify the Mixture Density Head
    head_config={},  # Number of components in the mixture
    embedding_dims=[],              # Use default inferred embeddings
    embedding_dropout=0.0,          # No dropout in embeddings
    batch_norm_continuous_input=True,  # Normalize continuous inputs
    learning_rate=1e-3,             # Learning rate
    metrics=["accuracy", "f1", "precision", "recall", "auroc"],  # Metrics to track
    metrics_params=[{}, {}, {}, {}, {"task": "multiclass"}],  # Metrics parameters
    metrics_prob_input=[True, True, True, True, True],  # Metrics expect probabilities
    seed=42                         # Seed for reproducibility
)

mdn_optimizer_config = OptimizerConfig()

mdn_trainer_config = TrainerConfig(
    auto_lr_find=False,
    batch_size=512,
    max_epochs=10
)

In [ ]:
from pytorch_tabular import TabularModel

tabular_model = TabularModel(
    data_config=mdn_data_config,
    model_config=mixture_density_config,
    optimizer_config=mdn_optimizer_config,
    trainer_config=mdn_trainer_config,
)

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off


In [ ]:
# Thay loss function ở đây
tabular_model.fit(train=train, validation=val, loss=DiceBCELoss())

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for classification task
INFO:pytorch_tabular.tabular_model:Preparing the Model: MDNModel


AssertionError: MDN is only implemented for Regression

In [ ]:
pred_df = tabular_model.predict(test)
pred_df.head()

AttributeError: 'TabularModel' object has no attribute 'model'

In [ ]:
result = tabular_model.evaluate(test)

### AutoInt(Automatic Feature Interaction Learning via Self-Attentive Neural Networks) (Huyền)

In [ ]:
from pytorch_tabular.models import AutoIntConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

autoint_data_config = DataConfig(
    target=[
        target_col
    ],  # target should always be a list
    continuous_cols=num_col_names.columns.tolist(),
    categorical_cols=cat_col_names,
)


autoint_config = AutoIntConfig(
    attn_embed_dim=32,            # Hidden units in Multi-Head Attention layers
    num_heads=2,                  # Number of attention heads
    num_attn_blocks=3,            # Number of attention layers
    attn_dropouts=0.1,            # Dropout between attention layers
    has_residuals=True,           # Enable residual connections
    embedding_dim=13,             # Embedding dimensions for features
    embedding_initialization="kaiming_normal",  # Embedding initialization scheme
    embedding_bias=True,          # Include bias in embedding layers
    share_embedding=False,        # Disable shared embeddings
    deep_layers=True,             # Enable deep MLP layers
    layers="13-13-13",           # Deep MLP layer configuration
    activation="ReLU",            # Activation function in deep MLP
    use_batch_norm=True,          # Include BatchNorm after layers
    dropout=0.1,                  # Dropout probability
    attention_pooling=False,      # Combine attention outputs for final prediction
    task="classification",        # Task type: regression or classification
    head="LinearHead",            # Prediction head type
    learning_rate=1e-3,           # Learning rate
    metrics=["accuracy", "f1_score", "precision", "recall", "auroc"],  # Metrics to track
    # metrics_params=[{}, {}, {}, {}, {"task": "multiclass"}],  # Metrics parameters
    metrics_prob_input=[True, True, True, True, True],  # Metrics expect probabilities
    seed=42,                      # Random seed for reproducibility
)

autoint_optimizer = OptimizerConfig()

autoint_trainer_config = TrainerConfig(
    auto_lr_find=False,
    batch_size=512,
    max_epochs=100
)

In [ ]:
from pytorch_tabular import TabularModel

tabular_model = TabularModel(
    data_config=autoint_data_config,
    model_config=autoint_config,
    optimizer_config=autoint_optimizer,
    trainer_config=autoint_trainer_config,
)

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off


In [ ]:
# DiceLoss()
# DiceBCELoss()
# IoULoss()
# FocalLoss()
# TverskyLoss()
# FocalTverskyLoss()

'''
Đổi loss function ở đây
Test với trường hợp không sử dụng custom loss function(ko cần truyền tham số)
'''
tabular_model.fit(train=train, validation=val, loss=FocalTverskyLoss())
# tabular_model.fit(train=train, validation=val)

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for classification task
INFO:pytorch_tabular.tabular_model:Preparing the Model: AutoIntModel
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_tabular.tabular_model:Training Started
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/saved_models exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ custom_loss      │ FocalTverskyLoss │      0 │ train │
│ 1 │ _backbone        │ AutoIntBackbone  │ 14.2 K │ train │
│ 2 │ _embedding_layer │ Embedding2dLayer │    364 │ train │
│ 3 │ _head            │ LinearHead       │    834 │ train │
└───┴──────────────────┴──────────────────┴────────┴───────┘

Trainable params: 15.4 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 15.4 K                                                                                               
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 35                                                                                          
Modules in eval mode: 0

Output()

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (4)
is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you 
want to see logs for the training epoch.

INFO:pytorch_tabular.tabular_model:Training the model completed
INFO:pytorch_tabular.tabular_model:Loading the best model
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/utils/python_utils.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please op

In [ ]:
pred_df = tabular_model.predict(test)
pred_df.head()

,R/Not-R_0.0_probability,R/Not-R_1.0_probability,R/Not-R_prediction
856,0.488600,0.511400,1.0
1867,0.333472,0.666528,1.0
651,0.127185,0.872815,1.0
368,0.411439,0.588561,1.0
2465,0.394915,0.605085,1.0


In [ ]:
result = tabular_model.evaluate(test)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5371428728103638     │
│        test_auroc         │    0.5993543863296509     │
│       test_f1_score       │    0.5371428728103638     │
│         test_loss         │    0.3325091004371643     │
│        test_loss_0        │    0.3325091004371643     │
│      test_precision       │    0.5371428728103638     │
│        test_recall        │    0.5371428728103638     │
└───────────────────────────┴───────────────────────────┘

### TabTransformer (Khoa)

In [ ]:
from pytorch_tabular.models import TabTransformerConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

tab_transformer_data_config = DataConfig(
    target=[
        target_col
    ],  # target should always be a list
    continuous_cols=num_col_names.columns.tolist(),
    categorical_cols=cat_col_names,
)

tab_transformer_config = TabTransformerConfig(
    input_embed_dim=32,                # Embedding dimension for categorical features
    embedding_initialization="kaiming_normal",  # Embedding initialization scheme
    embedding_bias=False,              # Disable embedding bias
    share_embedding=False,             # Disable shared embeddings
    num_heads=8,                       # Number of attention heads
    num_attn_blocks=6,                 # Number of transformer blocks
    transformer_head_dim=None,         # Default to same as `input_embed_dim`
    attn_dropout=0.1,                  # Dropout after Multi-Headed Attention
    add_norm_dropout=0.1,              # Dropout in AddNorm layers
    ff_dropout=0.1,                    # Dropout in Feed-Forward layers
    ff_hidden_multiplier=4,           # Multiplier for feed-forward hidden layers
    transformer_activation="GEGLU",    # Activation function in transformer layers
    task="classification",             # Task type: regression or classification
    head="LinearHead",                 # Prediction head type
    learning_rate=1e-3,                # Learning rate
    # loss="cross_entropy",              # Loss function for classification
    metrics=["accuracy", "f1_score", "precision", "recall", "auroc"],  # Metrics to track
    # metrics_params=[{}, {}, {}, {}, {"task": "multiclass"}],  # Metrics parameters
    metrics_prob_input=[True, True, True, True, True],  # Metrics expect probabilities
    seed=42                            # Random seed for reproducibility
)

tab_transformer_optimizer = OptimizerConfig()

tab_transformer_trainer_config = TrainerConfig(
    auto_lr_find=False,
    batch_size=512,
    max_epochs=100
)

In [ ]:
from pytorch_tabular import TabularModel

tabular_model = TabularModel(
    data_config=tab_transformer_data_config,
    model_config=tab_transformer_config,
    optimizer_config=tab_transformer_optimizer,
    trainer_config=tab_transformer_trainer_config
)

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off


In [ ]:
# DiceLoss()
# DiceBCELoss()
# IoULoss()
# FocalLoss()
# TverskyLoss()
# FocalTverskyLoss()

'''
Đổi loss function ở đây
Test với trường hợp không sử dụng custom loss function(ko cần truyền tham số)
'''
# tabular_model.fit(train=train, validation=val, loss=FocalTverskyLoss())
tabular_model.fit(train=train, validation=val)

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for classification task
INFO:pytorch_tabular.tabular_model:Preparing the Model: TabTransformerModel
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_tabular.tabular_model:Training Started
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/saved_models exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type                   ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ _backbone        │ TabTransformerBackbone │  271 K │ train │
│ 1 │ _embedding_layer │ Embedding2dLayer       │      0 │ train │
│ 2 │ _head            │ LinearHead             │     28 │ train │
│ 3 │ loss             │ CrossEntropyLoss       │      0 │ train │
└───┴──────────────────┴────────────────────────┴────────┴───────┘

Trainable params: 271 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 271 K                                                                                                
Total estimated model params size (MB): 1                                                                          
Modules in train mode: 119                                                                                         
Modules in eval mode: 0

Output()

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (4)
is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you 
want to see logs for the training epoch.

In [ ]:
pred_df = tabular_model.predict(test)
pred_df.head()

,R/Not-R_0.0_probability,R/Not-R_1.0_probability,R/Not-R_prediction
856,0.373352,0.626648,1.0
1867,0.596490,0.403510,0.0
651,0.330578,0.669422,1.0
368,0.394986,0.605014,1.0
2465,0.553807,0.446193,0.0


In [ ]:
result = tabular_model.evaluate(test)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5638095140457153     │
│        test_auroc         │    0.5389694571495056     │
│       test_f1_score       │    0.5638095140457153     │
│         test_loss         │    0.7367742657661438     │
│        test_loss_0        │    0.7367742657661438     │
│      test_precision       │    0.5638095140457153     │
│        test_recall        │    0.5638095140457153     │
└───────────────────────────┴───────────────────────────┘

### Gated Additive Tree Ensemble (Khoa)

In [ ]:
from pytorch_tabular.models import GatedAdditiveTreeEnsembleConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

gate_data_config = DataConfig(
    target=[
        target_col
    ],  # target should always be a list
    continuous_cols=num_col_names.columns.tolist(),
    categorical_cols=cat_col_names,
)

gate_config = GatedAdditiveTreeEnsembleConfig(
    gflu_stages=6,                       # Number of feature abstraction layers
    gflu_dropout=0.1,                    # Dropout rate for abstraction layers
    tree_depth=5,                        # Depth of each tree
    num_trees=20,                        # Number of trees in the ensemble
    binning_activation="sparsemoid",     # Activation function for binning
    feature_mask_function="sparsemax",   # Feature mask function
    tree_dropout=0.1,                    # Dropout probability in tree binning
    chain_trees=True,                    # Chain trees (boosting) or bag trees (parallel)
    tree_wise_attention=True,            # Enable tree-wise attention
    tree_wise_attention_dropout=0.1,     # Dropout in tree-wise attention
    share_head_weights=True,             # Share weights across heads
    task="classification",               # Task type: regression or classification
    head="LinearHead",                   # Prediction head type
    learning_rate=1e-3,                  # Learning rate
    metrics=["accuracy", "f1_score", "precision", "recall", "auroc"],  # Metrics to track
    # metrics_params=[{}, {}, {}, {}, {"task": "multiclass"}],  # Metrics parameters
    metrics_prob_input=[True, True, True, True, True],  # Metrics expect probabilities
    seed=42                              # Seed for reproducibility
)

gate_optimizer_config = OptimizerConfig()

gate_trainer_config = TrainerConfig(
    auto_lr_find=False,
    batch_size=512,
    max_epochs=100
)

In [ ]:
from pytorch_tabular import TabularModel

tabular_model = TabularModel(
    data_config=gate_data_config,
    model_config=gate_config,
    optimizer_config=gate_optimizer_config,
    trainer_config=gate_trainer_config,
)

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off


In [ ]:
# DiceLoss()
# DiceBCELoss()
# IoULoss()
# FocalLoss()
# TverskyLoss()
# FocalTverskyLoss()

'''
Đổi loss function ở đây
Test với trường hợp không sử dụng custom loss function(ko cần truyền tham số)
'''
tabular_model.fit(train=train, validation=val, loss=DiceBCELoss())

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for classification task
INFO:pytorch_tabular.tabular_model:Preparing the Model: GatedAdditiveTreeEnsembleModel
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_tabular.tabular_model:Training Started
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/saved_models exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type                       ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ custom_loss      │ DiceBCELoss                │      0 │ train │
│ 1 │ _backbone        │ GatedAdditiveTreesBackbone │  1.0 M │ train │
│ 2 │ _embedding_layer │ Embedding1dLayer           │     26 │ train │
│ 3 │ _head            │ CustomHead                 │     86 │ train │
└───┴──────────────────┴────────────────────────────┴────────┴───────┘

Trainable params: 1.0 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.0 M                                                                                                
Total estimated model params size (MB): 4                                                                          
Modules in train mode: 689                                                                                         
Modules in eval mode: 0

Output()

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (4)
is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you 
want to see logs for the training epoch.

INFO:pytorch_tabular.tabular_model:Training the model completed
INFO:pytorch_tabular.tabular_model:Loading the best model
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/utils/python_utils.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please op

In [ ]:
pred_df = tabular_model.predict(test)
pred_df.head()

,R/Not-R_0.0_probability,R/Not-R_1.0_probability,R/Not-R_prediction
856,0.583154,0.416846,0.0
1867,0.564480,0.435520,0.0
651,0.558963,0.441037,0.0
368,0.587972,0.412028,0.0
2465,0.592638,0.407362,0.0


In [ ]:
result = tabular_model.evaluate(test)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5066666603088379     │
│        test_auroc         │    0.4564528167247772     │
│       test_f1_score       │    0.5066666603088379     │
│         test_loss         │    1.1765083074569702     │
│        test_loss_0        │    1.1765083074569702     │
│      test_precision       │    0.5066666603088379     │
│        test_recall        │    0.5066666603088379     │
└───────────────────────────┴───────────────────────────┘

### Gated Adaptive Network for Deep Automated Learning of Features (GANDALF) (Huỳnh)

In [ ]:
from pytorch_tabular.models import GANDALFConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

gandaf_data_config = DataConfig(
    target=[
        target_col
    ],  # target should always be a list
    continuous_cols=num_col_names.columns.tolist(),
    categorical_cols=cat_col_names,
)

gandaf_config = GANDALFConfig(
    task="classification",
    gflu_stages=6,
    gflu_feature_init_sparsity=0.3,
    gflu_dropout=0.0,
    learning_rate=1e-3,
    metrics=["accuracy", "f1_score", "precision", "recall", "auroc"],  # Metrics to track
    # metrics_params=[{}, {}, {}, {}, {"task": "multiclass"}],  # Metrics parameters
    metrics_prob_input=[True, True, True, True, True],  # Metrics expect probabilities
)

gandaf_optimizer_config = OptimizerConfig()

gandaf_trainer_config = TrainerConfig(
    batch_size=1024,
    max_epochs=100,
)


In [ ]:
from pytorch_tabular import TabularModel

tabular_model = TabularModel(
    data_config=gandaf_data_config,
    model_config=gandaf_config,
    optimizer_config=gandaf_optimizer_config,
    trainer_config=gandaf_trainer_config,
    verbose=True
)

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off


In [ ]:
# DiceLoss()
# DiceBCELoss()
# IoULoss()
# FocalLoss()
# TverskyLoss()
# FocalTverskyLoss()

'''
Đổi loss function ở đây
Test với trường hợp không sử dụng custom loss function(ko cần truyền tham số)
'''
# tabular_model.fit(train=train, validation=val, loss=FocalTverskyLoss())
tabular_model.fit(train=train, validation=val)

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for classification task
INFO:pytorch_tabular.tabular_model:Preparing the Model: GANDALFModel
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_tabular.tabular_model:Training Started
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/saved_models exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ _backbone        │ GANDALFBackbone  │  6.4 K │ train │
│ 1 │ _embedding_layer │ Embedding1dLayer │     26 │ train │
│ 2 │ _head            │ Sequential       │     30 │ train │
│ 3 │ loss             │ CrossEntropyLoss │      0 │ train │
└───┴──────────────────┴──────────────────┴────────┴───────┘

Trainable params: 6.5 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 6.5 K                                                                                                
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 27                                                                                          
Modules in eval mode: 0

Output()

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (2)
is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you 
want to see logs for the training epoch.

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


INFO:pytorch_tabular.tabular_model:Training the model completed
INFO:pytorch_tabular.tabular_model:Loading the best model
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/utils/python_utils.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please op

In [ ]:
pred_df = tabular_model.predict(test)
pred_df.head()

,R/Not-R_0.0_probability,R/Not-R_1.0_probability,R/Not-R_prediction
856,0.736467,0.263533,0.0
1867,0.455912,0.544088,1.0
651,0.997221,0.002779,0.0
368,0.267918,0.732082,1.0
2465,0.332331,0.667669,1.0


In [ ]:
result = tabular_model.evaluate(test)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7580952644348145     │
│        test_auroc         │    0.8379611968994141     │
│       test_f1_score       │    0.7580952644348145     │
│         test_loss         │    0.4724096953868866     │
│        test_loss_0        │    0.4724096953868866     │
│      test_precision       │    0.7580952644348145     │
│        test_recall        │    0.7580952644348145     │
└───────────────────────────┴───────────────────────────┘

### DANETs: Deep Abstract Networks for Tabular Data Classification (Huỳnh)

In [ ]:
from pytorch_tabular.models import DANetConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

danet_data_config = DataConfig(
    target=[
        target_col
    ],  # target should always be a list
    continuous_cols=num_col_names.columns.tolist()
)

danet_config = DANetConfig(
    n_layers=8,                           # Number of Blocks in the DANet
    abstlay_dim_1=32,                     # Dimension of intermediate output in the first ABSTLAY layer
    abstlay_dim_2=64,                     # Dimension of intermediate output in the second ABSTLAY layer
    k=5,                                  # Number of feature groups in ABSTLAY layer
    dropout_rate=0.1,                     # Dropout in the Block
    task="classification",                # Task type: regression or classification
    head="LinearHead",                    # Prediction head type
    learning_rate=1e-3,                   # Learning rate
    metrics=["accuracy", "f1_score", "precision", "recall", "auroc"],  # Metrics to track
    # metrics_params=[{}, {}, {}, {}, {"task": "multiclass"}],  # Metrics parameters
    metrics_prob_input=[True, True, True, True, True],  # Metrics expect probabilities
    seed=42                               # Seed for reproducibility
)

danet_optimizer = OptimizerConfig()

danet_trainer_config = TrainerConfig(
    auto_lr_find=False,
    batch_size=512,
    max_epochs=100
)


In [ ]:
from pytorch_tabular import TabularModel

tabular_model = TabularModel(
    data_config=danet_data_config,
    model_config=danet_config,
    optimizer_config=danet_optimizer,
    trainer_config=danet_trainer_config,
)

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off


In [ ]:
# DiceLoss()
# DiceBCELoss()
# IoULoss()
# FocalLoss()
# TverskyLoss()
# FocalTverskyLoss()

'''
Đổi loss function ở đây
Test với trường hợp không sử dụng custom loss function(ko cần truyền tham số)
'''
# tabular_model.fit(train=train, validation=val, loss=FocalTverskyLoss())
tabular_model.fit(train=train, validation=val)

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for classification task
INFO:pytorch_tabular.tabular_model:Preparing the Model: DANetModel
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_tabular.tabular_model:Training Started
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/saved_models exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ _backbone        │ DANetBackbone    │  420 K │ train │
│ 1 │ _embedding_layer │ Embedding1dLayer │     26 │ train │
│ 2 │ _head            │ LinearHead       │    130 │ train │
│ 3 │ loss             │ CrossEntropyLoss │      0 │ train │
└───┴──────────────────┴──────────────────┴────────┴───────┘

Trainable params: 420 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 420 K                                                                                                
Total estimated model params size (MB): 1                                                                          
Modules in train mode: 156                                                                                         
Modules in eval mode: 0

Output()

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (4)
is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you 
want to see logs for the training epoch.

INFO:pytorch_tabular.tabular_model:Training the model completed
INFO:pytorch_tabular.tabular_model:Loading the best model
/usr/local/lib/python3.10/dist-packages/pytorch_tabular/utils/python_utils.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please op

In [ ]:
pred_df = tabular_model.predict(test)
pred_df.head()

,R/Not-R_0.0_probability,R/Not-R_1.0_probability,R/Not-R_prediction
856,0.588891,0.411109,0.0
1867,0.526263,0.473737,0.0
651,0.761764,0.238236,0.0
368,0.535175,0.464825,0.0
2465,0.467824,0.532176,1.0


In [ ]:
result = tabular_model.evaluate(test)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5257142782211304     │
│        test_auroc         │    0.6027570962905884     │
│       test_f1_score       │    0.5257142782211304     │
│         test_loss         │    0.6931993365287781     │
│        test_loss_0        │    0.6931993365287781     │
│      test_precision       │    0.5257142782211304     │
│        test_recall        │    0.5257142782211304     │
└───────────────────────────┴───────────────────────────┘

# Note

In [ ]:
'''
- Khởi tạo model (Done)
- Thử nghiệm các model với dữ liệu mặc định (Done)
- Thử nghiệm các model với dữ liệu custom (Done)

Các thuật toán:
- NODE
- TabNet
- AutoInt
- TabTransformer
- Gated Additive Tree Ensemble
- GANDALF
- DANET

'''